# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities = pd.read_csv('cities.csv')
cities.head()
cities

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Atuona,3,PF,1597780296,74,-9.80,-139.03,79.20,21.50
1,1,Te Anau,100,NZ,1597780296,89,-45.42,167.72,42.94,2.10
2,2,Mataura,100,NZ,1597780297,81,-46.19,168.86,43.84,5.08
3,3,Rikitea,0,PF,1597780297,79,-23.12,-134.97,72.05,10.85
4,4,Saint George,1,US,1597780297,13,37.10,-113.58,106.00,6.17
...,...,...,...,...,...,...,...,...,...,...
547,547,Sibolga,100,ID,1597780390,79,1.74,98.78,77.41,2.75
548,548,Churapcha,0,RU,1597780391,64,62.00,132.43,42.82,4.50
549,549,Ngorongoro,46,TZ,1597780391,64,-3.24,35.49,52.75,6.33
550,550,Okitipupa,0,NG,1597780391,92,6.50,4.78,73.51,4.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
humidity = cities['Humidity'].astype(float)
max_humidity = humidity.max()
locats = cities[['Lat', 'Lng']]

In [4]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locats, weights=humidity,dissipating=False, max_intensity=max_humidity,point_radius=3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
poss_cities_df = cities.loc[(cities['Max Temp'] > 70) & (cities['Max Temp'] < 80) & (cities['Cloudiness'] == 0), :]
poss_cities_df = poss_cities_df.dropna(how='any')
poss_cities_df.reset_index(inplace=True)
del poss_cities_df['index']
hotel_df = poss_cities_df
hotel_df.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,3,Rikitea,0,PF,1597780297,79,-23.12,-134.97,72.05,10.85
1,27,Faanui,0,PF,1597780202,78,-16.48,-151.75,78.30,18.34
2,122,Safi,0,MA,1597780318,79,32.30,-9.24,72.01,19.80
3,124,Malanje,0,AO,1597780319,45,-9.54,16.34,70.65,6.91
4,168,Lata,0,PT,1597780326,67,40.16,-8.33,73.99,5.99


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotellist = []

for i in range(len(hotel_df)):
    
    lat = hotel_df.loc[i]['Lat']
    lng = hotel_df.loc[i]['Lng']
    
    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key
    }

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    request = requests.get(base_url, params=params)
    json = request.json()
    try:
        hotellist.append(jsn['results'][0]['name'])
    except:
        hotellist.append('')
hotel_df['Hotel Name'] = hotellist
hotel_df = hotel_df.dropna(how='any')
hotel_df.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,3,Rikitea,0,PF,1597780297,79,-23.12,-134.97,72.05,10.85,
1,27,Faanui,0,PF,1597780202,78,-16.48,-151.75,78.30,18.34,
2,122,Safi,0,MA,1597780318,79,32.30,-9.24,72.01,19.80,
3,124,Malanje,0,AO,1597780319,45,-9.54,16.34,70.65,6.91,
4,168,Lata,0,PT,1597780326,67,40.16,-8.33,73.99,5.99,


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locats)
fig.add_layer(markers)
fig


Figure(layout=FigureLayout(height='420px'))